In [2]:
import logging

# praw logger의 레벨을 WARNING 이상으로 차단
logging.getLogger("praw").setLevel(logging.ERROR)


In [1]:
!pip install praw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 4.3 MB/s eta 0:00:00


In [3]:
import praw
import pandas as pd
from datetime import datetime
import time

# 🔐 Reddit API 인증
reddit = praw.Reddit(
    client_id="H1ollC1ls5_hwWKFRO8SPQ",          # Reddit 앱에서 복사
    client_secret="yJ2f2eI6nOS-gj4IqmfrTkZOIaZwgg",  # Reddit 앱에서 복사
    user_agent="m7_sentiment_bot",       # 자유롭게 지정
    username="Upper_Article8633",     # Reddit 계정 ID
    password="jjh851070!"      # Reddit 계정 비밀번호
)

# M7 종목, 서브레딧, 연도별 설정
m7_stocks = ["AAPL", "MSFT", "NVDA", "TSLA", "AMZN", "META", "GOOGL"]
subreddits = ["stocks", "wallstreetbets"]
years = [2020, 2021, 2022, 2023, 2024, 2025]

results = []

# 수집 시작
for stock in m7_stocks:
    for subreddit in subreddits:
        for year in years:
            print(f"Collecting: {stock} | r/{subreddit} | {year}")
            query = stock
            subreddit_instance = reddit.subreddit(subreddit)

            try:
                # Reddit 검색 제한 우회: time filter 없이 최신 게시글 검색 후 필터링
                for post in subreddit_instance.search(query, sort="new", time_filter="all", limit=2000):
                    created_time = datetime.fromtimestamp(post.created_utc)
                    if created_time.year == year:
                        results.append({
                            "stock": stock,
                            "subreddit": subreddit,
                            "year": year,
                            "title": post.title,
                            "text": post.selftext,
                            "score": post.score,
                            "num_comments": post.num_comments,
                            "created": created_time.strftime("%Y-%m-%d %H:%M")
                        })
                    time.sleep(0.2)  # Reddit 요청 제한 방지용
            except Exception as e:
                print(f"오류 발생: {e}")
                continue

# 🔽 DataFrame 저장
df = pd.DataFrame(results)
# df.to_csv("m7_reddit_2020_2024_praw.csv", index=False)
print(f"\n✅ 총 {len(df)}개의 Reddit 게시글 수집 완료")
df

Collecting: AAPL | r/stocks | 2020
Collecting: AAPL | r/stocks | 2021
Collecting: AAPL | r/stocks | 2022
Collecting: AAPL | r/stocks | 2023
Collecting: AAPL | r/stocks | 2024
Collecting: AAPL | r/stocks | 2025
Collecting: AAPL | r/wallstreetbets | 2020
Collecting: AAPL | r/wallstreetbets | 2021
Collecting: AAPL | r/wallstreetbets | 2022
Collecting: AAPL | r/wallstreetbets | 2023
Collecting: AAPL | r/wallstreetbets | 2024
Collecting: AAPL | r/wallstreetbets | 2025
Collecting: MSFT | r/stocks | 2020
Collecting: MSFT | r/stocks | 2021
Collecting: MSFT | r/stocks | 2022
Collecting: MSFT | r/stocks | 2023
Collecting: MSFT | r/stocks | 2024
Collecting: MSFT | r/stocks | 2025
Collecting: MSFT | r/wallstreetbets | 2020
Collecting: MSFT | r/wallstreetbets | 2021
Collecting: MSFT | r/wallstreetbets | 2022
Collecting: MSFT | r/wallstreetbets | 2023
Collecting: MSFT | r/wallstreetbets | 2024
Collecting: MSFT | r/wallstreetbets | 2025
Collecting: NVDA | r/stocks | 2020
Collecting: NVDA | r/stocks |

,stock,subreddit,year,title,text,score,num_comments,created
0,AAPL,stocks,2023,"Even Though QQQ works, it pisses me off",If I were to tell you I came up with an idea t...,159,73,2023-12-30 14:48
1,AAPL,stocks,2023,r/Stocks Daily Discussion & Fundamentals Frida...,"This is the daily discussion, so anything stoc...",12,197,2023-12-29 10:30
2,AAPL,stocks,2023,"Whats your ""this could be the next tsla or aap...",Just like the title says.... what are people's...,274,545,2023-12-27 15:19
3,AAPL,stocks,2023,r/Stocks Daily Discussion & Fundamentals Frida...,"This is the daily discussion, so anything stoc...",12,148,2023-12-22 10:30
4,AAPL,stocks,2023,"Thoughts on selling covered calls - AAPL, AMD,...",I’ve sold covered calls in higher volatility s...,0,8,2023-12-20 16:13
...,...,...,...,...,...,...,...,...
3358,GOOGL,wallstreetbets,2025,Rumors of HUMA's demise are greatly exaggerated,This DD on HUMA requires the reader to be a bi...,114,64,2025-01-10 10:11
3359,GOOGL,wallstreetbets,2025,Meta ⭐️5,Playing the TikTok ban. Posting for the 4th ti...,12,15,2025-01-08 02:11
3360,GOOGL,wallstreetbets,2025,Alphabet to bring Gemini AI to TV sets running...,,74,15,2025-01-07 02:21
3361,GOOGL,wallstreetbets,2025,Am I doing the is right? Full regard,,111,44,2025-01-02 18:21


In [11]:
df.to_csv("m7_reddit_2020_2025_praw.csv", index=False)

In [17]:
print(f"The number of data in 2020: {len(df[df['year'] == 2020])}")
print(f"The number of data in 2021: {len(df[df['year'] == 2021])}")
print(f"The number of data in 2022: {len(df[df['year'] == 2022])}")
print(f"The number of data in 2023: {len(df[df['year'] == 2023])}")
print(f"The number of data in 2024: {len(df[df['year'] == 2024])}")
print(f"The number of data in 2025: {len(df[df['year'] == 2025])}")

The number of data in 2020: 0
The number of data in 2021: 0
The number of data in 2022: 32
The number of data in 2023: 256
The number of data in 2024: 1851
The number of data in 2025: 1224
